#### (I). Inference with Phi-3 on 163 HumanEval cases
* Error / Success with Message information included

In [1]:
from sample_hf import CodeCompletionGenerator
from human_eval.data import write_jsonl, read_problems
from tqdm import tqdm

model_name = "microsoft/Phi-3-mini-128k-instruct"
# model_name="meta-llama/Meta-Llama-3-8B"
is_instruct = True
file_name = "eval_phi_3_instruct"

code_gen = CodeCompletionGenerator(model_name=model_name, device="cpu", is_instruct=is_instruct)

problems = read_problems()
problems = {k: problems[k] for k in list(problems.keys())[:1]} # SubSample Problems out of the 163 cases


num_samples_per_task = 1
samples = [
    dict(task_id=task_id, completion=code_gen.generate_one_completion(problems[task_id]["prompt"]))
    for task_id in problems
    for _ in tqdm(range(num_samples_per_task))
]
write_jsonl(f"{file_name}.jsonl", samples)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 1/1 [00:35<00:00, 35.30s/it]


In [2]:
from human_eval.data import stream_jsonl
from sample_hf import check_code

sample_file = f"{file_name}.jsonl"
for sample in stream_jsonl(sample_file): # This is a generator
    task_id = sample['task_id']
    problem = problems[task_id]
    success, message = check_code(task_id, problem, sample['completion'])
    print("Success: ", success, " || Message: ", message)


Success:  True  || Message:  
